In [106]:
import pprint
import json
import time
from selenium.webdriver.common.action_chains import ActionChains
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from datetime import date


options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
DRIVER_PATH = '/usr/bin/chromedriver'
driver = webdriver.Chrome(options = options, executable_path=DRIVER_PATH)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

ua = UserAgent()
a = ua.random
user_agent = ua.random

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent":user_agent})

print(user_agent)

Mozilla/5.0 (Windows NT 6.2; rv:22.0) Gecko/20130405 Firefox/22.0


PARAMS

In [107]:
url = "https://www.lamudi.com.mx/"
estado = "Estado de Mexico"
minprice = "1200000"
maxprice = "1700000"
tipo = "Otros"

In [108]:
driver.get(url)
wait = WebDriverWait(driver,100)

text_box = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "SearchBar-searchField")))
text_box.send_keys(Keys.CONTROL + "a")
text_box.send_keys(Keys.DELETE)
text_box.send_keys(estado)
text_box.send_keys(Keys.ENTER)

button_go = driver.find_element_by_class_name("SearchBar-button")
button_go.click()

minprice_box = wait.until(ec.presence_of_element_located((By.NAME, "rangeInputMin")))
maxprice_box = wait.until(ec.presence_of_element_located((By.NAME, "rangeInputMax")))

minprice_box.send_keys(Keys.DELETE)
minprice_box.send_keys(Keys.CONTROL + "a")
minprice_box.send_keys(minprice)
maxprice_box.send_keys(Keys.CONTROL + "a")
maxprice_box.send_keys(Keys.DELETE)
maxprice_box.send_keys(maxprice)

button_go = driver.find_element_by_class_name("UiButton")
button_go.click()

In [109]:
def getTextbyClass(driver, strElement):
    text = 'NA'
    try:
        text = driver.find_element_by_class_name(strElement)
        text = text.text
    except:
        print(f'Could not find element {strElement}')
    return text

def getTextByID(driver, strElement):
    text = 'NA'
    try:
        text = driver.find_element_by_id(strElement)
        text = text.text
    except:
        print(f'Could not find element {strElement}')
    return text  
    

In [110]:
def process_card(link, record):
    time.sleep(randint(3, 4))
    link.click()
    driver.switch_to.window(driver.window_handles[1])
    
    try:
        price = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "FirstPrice")))
        record["price"] = price.text
    except:
        price = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "SecondPrice")))
        record["price"] = price.text    
    
    record["title_header_sup"] = getTextbyClass(driver,"Header-title-block") 
    record["title_header_loc"] = getTextbyClass(driver,"Header-title-address-text")
    record["descripcion"] = getTextbyClass(driver, "ViewMore-text-description")  

        
    attributes = driver.find_elements_by_class_name("columns-2")
    
    for attribute in attributes:
        if "Terreno" in attribute.text:     
            record[f'terreno'] = getTextbyClass(attribute,"last")

        if "Construidos" in attribute.text:     
            record[f'Construidos'] = getTextbyClass(attribute,"last")
        
        if "Baños" in attribute.text:     
            record[f'Baños'] = getTextbyClass(attribute,"last")    
        
        if "Estacionamientos" in attribute.text:     
            record[f'Estacionamientos'] = getTextbyClass(attribute,"last")     
        
        if "Recámaras" in attribute.text:     
            record[f'Recámaras'] = getTextbyClass(attribute,"last")     
        
        if "Medio baño" in attribute.text:     
            record[f'Medio baño'] = getTextbyClass(attribute,"last")
        
        if "Construido" in attribute.text:     
            record[f'Antigüedad'] = getTextbyClass(attribute,"last")    
        
           
    try:
        mapSrc = driver.find_element_by_id("js-developmentMap")
        lat =  mapSrc.get_attribute("data-lat")
        lon = mapSrc.get_attribute("data-lng")
    except:
        lat = "0"
        lon = "0"
        
    record["lat"] = lat      
    record["lon"] = lon   
    record["date_processed"]  = date.today().strftime('%Y-%m-%d')
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [111]:
def processPage(pageNumber):
    list_container = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "js-listingContainer")))
    listing = list_container.find_elements_by_class_name("ListingCell-content")

    recordlist = []
    counter = 1

    for element in listing:
        
        record = {}
        
        idContainer = element.find_element_by_class_name("js-FavoriteButton")
        record["data-id"] = idContainer.get_attribute("data-id")
        idContainer = element.find_element_by_class_name("ListingCell-AllInfo")
        record["type"] = idContainer.get_attribute("data-category")   
        link = element.find_element_by_tag_name("a")
        record["link"] = link.get_attribute("href")
        
        recordlist.append(record)
        process_card(link, record)    
        print(f'ELEMENT {counter} PROCESSED SUCESSFULLY')
        print(f'---------------------------------------')
        counter += 1
        

    outfile = (f'data_{estado}_{pageNumber}.json')
    with open(outfile, 'w') as out:
        json.dump(recordlist, out,  sort_keys=True, indent=4)

In [ ]:
pageNumber = 1

while True:
    processPage(pageNumber)
    pageNumber = pageNumber + 1
    
    try:
        next_page = driver.find_element_by_class_name("icon-navigate-right")
        next_page.click()
        time.sleep(randint(6, 8))
    except Exception as e:
        print("All pages were processed")
        break
    

Could not find element Header-title-address-text
ELEMENT 1 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 2 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 3 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 4 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 5 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 6 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 7 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 8 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-ti

ELEMENT 13 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 14 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 15 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 16 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 17 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 18 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 19 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 20 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 21 PROCESSED SUC

Could not find element Header-title-address-text
ELEMENT 21 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 22 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 23 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 24 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 25 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 26 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 27 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 28 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element H

ELEMENT 28 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 29 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 30 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 1 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 2 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 3 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 4 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 5 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 6 PROCESSED SUCESSFUL

Could not find element Header-title-address-text
ELEMENT 6 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 7 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 8 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 9 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 10 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 11 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 12 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 13 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Heade

...

ELEMENT 20 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 21 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 22 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 23 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 24 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 25 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 1 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 2 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 3 PROCESSED SUCESS

Could not find element Header-title-address-text
ELEMENT 13 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 14 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 15 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 16 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 17 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 18 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 19 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 20 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element H

ELEMENT 5 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 6 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 7 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 8 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 9 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 10 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 11 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 12 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 13 PROCESSED SUCESSFU

Could not find element Header-title-address-text
ELEMENT 23 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 24 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 25 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 1 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 2 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 3 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 4 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 5 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header

ELEMENT 15 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 16 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 17 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 18 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 19 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element Header-title-address-text
ELEMENT 20 PROCESSED SUCESSFULLY
---------------------------------------
